## 爬虫示例

爬取：https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/Getting_started

In [1]:
import requests
from bs4 import BeautifulSoup

### 1. 发送网络请求

In [2]:
url = 'https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/Getting_started'

In [3]:
r = requests.get(url)

In [4]:
r.status_code

200

In [9]:
content = r.text

### 2. 解析HTML 文件

In [10]:
soup = BeautifulSoup(content)

### 3. 从网页中提取需要的内容

任务1：获取文章的标题

In [16]:
article = soup.find('article', attrs={'class': 'main-page-content'})

In [17]:
article

<article class="main-page-content" lang="zh-CN"><h1>开始学习 HTML</h1><div class="section-content"><ul class="prev-next">
<li><a class="button secondary" href="/zh-CN/docs/Learn/HTML/Introduction_to_HTML"><span class="button-wrap"> Overview: Introduction to HTML</span></a></li>
<li><a class="button secondary" href="/zh-CN/docs/Learn/HTML/Introduction_to_HTML/The_head_metadata_in_HTML"><span class="button-wrap"> 下一页  </span></a></li>
</ul>
<p>本文将从 HTML 最基础的部分讲起，对元素（Element）、属性（Attribute）以及可能涉及的一些重要术语进行介绍，并明确它们在语言中所处的位置。本文还会讲解 HTML 元素和页面的组织方式，以及其他一些重要的基本语言特性。学习的过程中，我们会使用 HTML 做一些好玩的事情。</p>
<div class="table-scroll"><table class="learn-box standard-table">
<tbody>
<tr>
<th scope="row">预备知识：</th>
<td>具备计算机基础知识，安装了<a href="/zh-CN/docs/Learn/Getting_started_with_the_web/Installing_basic_software">基础软件环境</a>，了解基本的<a href="/zh-CN/docs/Learn/Getting_started_with_the_web/Dealing_with_files">文件组织方法</a>。</td>
</tr>
<tr>
<th scope="row">学习目标：</th>
<td>熟悉 HTML 语言的基础知识，掌握几个 HTML 元素的基本用法。</td>
</tr>
</tbo

In [18]:
# 从 article 中查找标题
title = article.find('h1').text
print(title)

开始学习 HTML


In [21]:
# 从 article 中查找所有的副标题，即目录
subtitles = article.find_all('h2')
print(subtitles)

[<h2 id="什么是_html"><a href="#什么是_html" title="Permalink to 什么是 HTML?">什么是 HTML?</a></h2>, <h2 id="剖析一个_html_元素"><a href="#剖析一个_html_元素" title="Permalink to 剖析一个 HTML 元素">剖析一个 HTML 元素</a></h2>, <h2 id="属性"><a href="#属性" title="Permalink to 属性">属性</a></h2>, <h2 id="剖析_html_文档"><a href="#剖析_html_文档" title="Permalink to 剖析 HTML 文档">剖析 HTML 文档</a></h2>, <h2 id="实体引用：在_html_中包含特殊字符"><a href="#实体引用：在_html_中包含特殊字符" title="Permalink to 实体引用：在 HTML 中包含特殊字符">实体引用：在 HTML 中包含特殊字符</a></h2>, <h2 id="html_注释"><a href="#html_注释" title="Permalink to HTML 注释">HTML 注释</a></h2>, <h2 id="总结"><a href="#总结" title="Permalink to 总结">总结</a></h2>, <h2 id="本章目录"><a href="#本章目录" title="Permalink to 本章目录">本章目录</a></h2>]


In [22]:
subtitles = [subtitle.text for subtitle in subtitles]
print(subtitles)

['什么是 HTML?', '剖析一个 HTML 元素', '属性', '剖析 HTML 文档', '实体引用：在 HTML 中包含特殊字符', 'HTML 注释', '总结', '本章目录']


任务2: 获取下一页的链接

In [25]:
next_page_button = article.find('a', {'class': 'button secondary'})

In [31]:
next_page_button.attrs

{'class': ['button', 'secondary'],
 'href': '/zh-CN/docs/Learn/HTML/Introduction_to_HTML'}

In [32]:
next_page_url = 'https://developer.mozilla.org' + next_page_button.attrs['href']

In [33]:
next_page_url

'https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML'

任务3：获取下一页的标题、副标题下一页，重复直到没有下一页

In [8]:
def obtain_page(url):
    # 发送请求
    r = requests.get(url)
    # 解析DOM
    content = BeautifulSoup(r.text)
    # 提取内容
    
    ## 提取标题
    article = content.find('article', {'class': 'main-page-content'})
    h1 = article.find('h1').text
    ## 副标题
    h2 = [i.text for i in article.find_all('h2')]
    
    ## 下一页
    next_page_button = article.find_all('a', {'class': 'button secondary'})[-1]
    url = 'https://developer.mozilla.org' + next_page_button.attrs['href']
    return h1, h2, url

In [9]:
url = "https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/Getting_started"
while True:
    try:
        h1, h2, url = obtain_page(url)
        print(f"已爬取{h1}，该文章包含以下章节：{','.join(h2)}，下一个页面的地址为：{url}")
    except Exception as err:
        print(err)
        break

已爬取开始学习 HTML，该文章包含以下章节：什么是 HTML?,剖析一个 HTML 元素,属性,剖析 HTML 文档,实体引用：在 HTML 中包含特殊字符,HTML 注释,总结,本章目录，下一个页面的地址为：https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/The_head_metadata_in_HTML
已爬取<head>标签里有什么？Metadata-HTML 中的元数据，该文章包含以下章节：什么是 HTML <head> 标签,添加标题,元数据：<meta> 元素,在你的站点增加自定义图标,在 HTML 中应用 CSS 和 JavaScript,为文档设定主语言,总结,本章目录，下一个页面的地址为：https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/HTML_text_fundamentals
已爬取HTML 文字处理基础，该文章包含以下章节：基础：标题和段落,列表,重点强调,总结,本章目录，下一个页面的地址为：https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/Creating_hyperlinks
已爬取创建超链接，该文章包含以下章节：什么是超链接？,链接的解析,统一资源定位符（URL）与路径（path）快速入门,链接最佳实践,主动学习：创建一个导航菜单,电子邮件链接,小结,本章目录，下一个页面的地址为：https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/Advanced_text_formatting
已爬取高阶文字排版，该文章包含以下章节：描述列表,引用,缩略语,标记联系方式,上标和下标,展示计算机代码,标记时间和日期,总结,本章目录，下一个页面的地址为：https://developer.mozilla.org/zh-CN/docs/Learn/HTML/Introduction_to_HTML/Document_and_website_structure
已爬取文档与网站